# 1. Data Extraction

In this notebook we will extract our data and analyse it. For that purpose, we are importing from our library where we define the
```bcrp_dataframe``` dataframe. This function will allows us to use the API interface of the Central Bank of Reserve of Peru (BCRP) to automatically create a pandas dataframe with the necessary codes.

## 1.1 Libraries

We import the necessary libraries, including our own library in the modules file

In [1]:
# Warnings
import warnings
warnings.filterwarnings("ignore")

# Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from scipy import stats
from functools import reduce

# Statsmodels
import statsmodels.api as sm
import pmdarima as pmd
from pmdarima.arima import auto_arima
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VARResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL

# Machine Learning models
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, ElasticNet, ElasticNetCV, LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    median_absolute_error,
    r2_score,
    precision_score

)

from xgboost import XGBRegressor



In [2]:
# We import our own functions
import sys
sys.path.append('../../..')  # Move two levels up to the project root
from modules.functions import *

## 1.2 Extraction
We define our inputs and apply them the ```bcrp_dataframe``` function in order to obtain the pandas dataframe with the corresponding series

We define the following inputs:

    series     = the code of the series we are going to extract
    start_date = the starting date, when the BCRP starts using the interest rate as a policy measure
    end_date   = December 2019
    freq       = Monthly frequency

### df_1
We can now create the first dataframe with the ```bcrp_dataframe``` function. This dataframe contains all Consumer Price Index variables in monthly % change. This variables are then seasonally adjusted using the 

In [3]:
series     = ['PN01278PM', 'PN01280PM', 'PN01282PM', 'PN01271PM', 'PN09817PM','PN09816PM', 'PN01276PM', 'PN01313PM', 'PN01314PM',  
             'PN01315PM', 'PN09818PM','PN01286PM']
start_date = '2003-09'
end_date   = '2008-12'
freq       = 'Mensual'

In [4]:
df_1 = bcrp_dataframe( series , start_date , end_date , freq )
df_1.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Importado,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor
Fecha,,,,,,,,,,,,
2003-09-01,0.005689,0.205933,0.822993,0.558598,1.416850,1.024345,-0.017156,0.900901,0.223770,0.048987,1.140655,0.669709
2003-10-01,-0.040038,-0.035055,0.096153,0.049032,0.164730,0.193699,-0.070649,0.198413,-0.040003,-0.132041,0.183697,0.171749
2003-11-01,0.125742,0.243529,0.120950,0.167685,0.237966,0.256361,0.005988,0.396040,-0.082393,0.088930,0.166302,0.190676
2003-12-01,0.127343,0.594507,0.534926,0.563951,1.196907,0.898519,0.231768,0.986193,0.242505,0.078004,0.306233,0.649838
2004-01-01,-0.055834,0.265543,0.708509,0.537447,1.379067,1.132403,-0.141462,1.074219,0.076551,-0.081012,3.494166,0.545590


In [5]:
df_1 = get_trend(df_1)
df_1.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Importado,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor
Fecha,,,,,,,,,,,,
2003-09-01,0.091983,0.482534,0.550585,0.522457,1.151261,0.985896,-0.025115,1.075523,0.028053,0.028619,1.091708,0.709608
2003-10-01,0.093421,0.471092,0.512928,0.495609,1.082513,0.926523,-0.013523,0.996365,0.048218,0.030202,1.052162,0.675723
2003-11-01,0.094738,0.458591,0.475632,0.468532,1.013426,0.866951,-0.002180,0.917675,0.067534,0.031715,1.010531,0.640840
2003-12-01,0.095937,0.445053,0.438602,0.441178,0.943887,0.807104,0.008895,0.839410,0.085945,0.033198,0.966929,0.604961
2004-01-01,0.097020,0.430631,0.401824,0.413606,0.874035,0.747053,0.019742,0.761551,0.103573,0.034709,0.921898,0.568245


### df_2
We create the second dataframe with the ```bcrp_dataframe``` function. This dataframe contains rate variables. We use this variables in levels. It is not necessary to differentiate them.

In [6]:
series     = ['PN00493MM', 'PD04722MM']
start_date = '2003-09'
end_date   = '2008-12'
freq       = 'Mensual'

In [7]:
df_2 = bcrp_dataframe( series , start_date , end_date , freq )
df_2.head()

,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria
Fecha,,
2003-09-01,11.014920,2.75
2003-10-01,10.349944,2.75
2003-11-01,11.690608,2.50
2003-12-01,11.391178,2.50
2004-01-01,10.634030,2.50


### df_3
We create the third dataframe with the ```bcrp_dataframe``` function. This dataframe contains monetary variables. We differentiate those variables in order to get their monthly % change

In [8]:
series     = ['PN00495MM', 'PN06481IM', 'PN02125PM', 'PN01661XM','PN01662XM','PN01664XM','PN01660XM']
start_date = '2003-09'
end_date   = '2008-12'
freq       = 'Mensual'

In [9]:
df_3 = bcrp_dataframe( series , start_date , end_date , freq )
df_3 = df_3.pct_change()
df_3 = df_3.dropna()
df_3.head()

,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
Fecha,,,,,,,
2003-10-01,0.010225,0.005542,0.056953,0.016588,-0.027808,0.181232,0.065797
2003-11-01,0.011445,0.050288,-0.001674,0.115205,0.075366,0.017022,0.022715
2003-12-01,0.016607,-0.010532,-0.005608,0.037146,0.048843,0.051254,0.037815
2004-01-01,0.019490,0.036233,-0.005346,0.003546,0.062558,0.026780,0.061517
2004-02-01,0.017003,-0.005743,-0.010744,-0.019994,0.081887,0.087963,0.015738


In [10]:
df = df_1.join(df_2).join(df_3)
df.dropna(inplace=True)
df.head()

,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,...,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
Fecha,,,,,,,,,,,,,,,,,,,,,
2003-10-01,0.093421,0.471092,0.512928,0.495609,1.082513,0.926523,-0.013523,0.996365,0.048218,0.030202,...,0.675723,10.349944,2.75,0.010225,0.005542,0.056953,0.016588,-0.027808,0.181232,0.065797
2003-11-01,0.094738,0.458591,0.475632,0.468532,1.013426,0.866951,-0.002180,0.917675,0.067534,0.031715,...,0.640840,11.690608,2.50,0.011445,0.050288,-0.001674,0.115205,0.075366,0.017022,0.022715
2003-12-01,0.095937,0.445053,0.438602,0.441178,0.943887,0.807104,0.008895,0.839410,0.085945,0.033198,...,0.604961,11.391178,2.50,0.016607,-0.010532,-0.005608,0.037146,0.048843,0.051254,0.037815
2004-01-01,0.097020,0.430631,0.401824,0.413606,0.874035,0.747053,0.019742,0.761551,0.103573,0.034709,...,0.568245,10.634030,2.50,0.019490,0.036233,-0.005346,0.003546,0.062558,0.026780,0.061517
2004-02-01,0.097976,0.415491,0.365316,0.385898,0.804075,0.686922,0.030393,0.684102,0.120567,0.036283,...,0.530890,10.718295,2.50,0.017003,-0.005743,-0.010744,-0.019994,0.081887,0.087963,0.015738


In [11]:
df.tail()

,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,...,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
Fecha,,,,,,,,,,,,,,,,,,,,,
2008-08-01,0.469680,0.356039,0.690723,0.554025,0.666096,0.681738,0.394014,0.755351,0.365874,0.317970,...,0.702300,11.472876,6.25,0.005009,0.002124,-0.005870,-0.012197,-0.160701,-0.156075,-0.128561
2008-09-01,0.481624,0.336413,0.723117,0.565199,0.675909,0.680067,0.422250,0.754096,0.388973,0.335349,...,0.693072,10.787601,6.50,0.017596,-0.006157,-0.005640,-0.125494,-0.006954,-0.078231,-0.103871
2008-10-01,0.493295,0.316247,0.755818,0.576337,0.686014,0.678143,0.450790,0.752209,0.412633,0.352862,...,0.682759,11.862506,6.50,0.003966,-0.079794,-0.006094,-0.206207,-0.260785,-0.244475,-0.267189
2008-11-01,0.504673,0.295514,0.788926,0.587490,0.696555,0.676114,0.479560,0.749865,0.436787,0.370484,...,0.671225,10.084114,6.50,0.006653,-0.030157,-0.003081,0.021940,-0.090908,-0.102564,-0.253950
2008-12-01,0.515746,0.274156,0.822524,0.598682,0.707608,0.674121,0.508447,0.747301,0.461270,0.388144,...,0.658290,12.516334,6.50,-0.005561,0.007297,-0.003589,-0.012756,-0.010237,-0.077524,-0.287431


## 1.3 Data Inspection
We inspect the df. We first verify that all values are non-null. The, we apply the ```describe``` function to see the main variables.

In [12]:
df.isna().sum()

Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente                            0
Índice de precios Lima Metropolitana (var% mensual) - IPC Transables                            0
Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables                         0
Índice de precios Lima Metropolitana (var% mensual) - IPC                                       0
Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente                         0
Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía                   0
Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía               0
Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas                   0
Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas               0
Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas    0
Índice de precios Li

In [13]:
df.describe()

,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables,Índice de precios Lima Metropolitana (var% mensual) - IPC,Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía,Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas,Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas,...,Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor,Tasas de interés del Banco Central de Reserva - Tasa de Encaje,Tasas de interés del Banco Central de Reserva - Tasa de Referencia de la Política Monetaria,Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado,Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$),Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100),Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas),Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)
count,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,...,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000
mean,0.198784,0.259449,0.300723,0.283644,0.403698,0.394933,0.147978,0.401208,0.176596,0.140437,...,0.396560,10.049576,4.043651,0.017491,0.019377,0.001126,0.010535,0.011290,0.006659,0.010379
std,0.139387,0.137878,0.206480,0.167603,0.272238,0.258684,0.114085,0.291265,0.093903,0.088424,...,0.220017,1.244708,1.235839,0.009393,0.039341,0.015714,0.075197,0.092101,0.074405,0.093474
min,0.091585,0.054574,0.072914,0.065985,-0.011302,0.034079,-0.013523,0.042260,0.048218,0.030202,...,0.086645,8.266673,2.500000,-0.006819,-0.108487,-0.010744,-0.206207,-0.260785,-0.244475,-0.287431
25%,0.099467,0.133045,0.136300,0.140318,0.183697,0.165435,0.105624,0.143251,0.114130,0.078857,...,0.206385,9.205722,3.000000,0.012341,-0.003019,-0.004814,-0.025061,-0.038329,-0.039057,-0.027649
50%,0.116670,0.257501,0.209665,0.218746,0.327533,0.327496,0.111462,0.301928,0.163332,0.127392,...,0.309342,9.813459,4.500000,0.018803,0.016045,-0.001779,0.011722,0.023512,0.011058,0.020974
75%,0.285922,0.388527,0.441197,0.443402,0.622833,0.677128,0.134758,0.748583,0.196780,0.157557,...,0.640015,10.676162,4.875000,0.022540,0.040109,-0.000131,0.036577,0.061659,0.049089,0.073302
max,0.515746,0.471092,0.822524,0.598682,1.082513,0.926523,0.508447,0.996365,0.461270,0.388144,...,0.729516,13.856740,6.500000,0.038067,0.110080,0.081553,0.241254,0.277119,0.181232,0.160941


We have 195 observation ranging from ```2003-10-01``` to ```2019-12-01```. The mean of monthly % change of all CPI variables is around 0.2. The mean of the lacing rate and the interest rate is 10.7% and 3.67%, respectively. The three monetary variables have a small monthly % change, around 0.01 and 0.001 for Minimum Wage index. 

## 1.4 Data adjustment
We will rename the columns for easier identification of the variables. We will also create a new dataframe with the lags of the variables. 

In [14]:
# New column names
columns = {
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente': 'CPI Core',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Transables': 'CPI Tradable',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC No Transables': 'CPI Non-Tradable',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC': 'CPI',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC No Subyacente': 'CPI Non-Core',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Energía': 'CPI Food and Energy',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Sin Alimentos y Energía': 'CPI Excluding Food and Energy',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Alimentos y Bebidas': 'CPI Food and Beverages',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC sin Alimentos y Bebidas': 'CPI Excluding Food and Beverages',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Subyacente Sin Alimentos y Bebidas': 'CPI Core Excluding Food and Beverages',
    'Índice de precios Lima Metropolitana (var% mensual) - IPC Importado': 'CPI Imported',
    'Índice de precios Lima Metropolitana (var% mensual) - Índice de Precios al por Mayor': 'Wholesale Price Index',
    'Tasas de interés del Banco Central de Reserva  - Tasa de Encaje': 'Reserve Requirement Rate',
    'Tasas de interés del Banco Central de Reserva  - Tasa de Referencia de la Política Monetaria': 'Monetary Policy Rate',
    'Emisión primaria y multiplicador (millones S/) - Circulante Desestacionalizado': 'Circulating Currency Seasonally Adjusted (mill S/)',
    'Liquidez internacional del BCRP - RIN - Reservas Internacionales Netas (millones US$)': 'Net International Reserves (mill $)',
    'Remuneraciones - Remuneración Mínima Vital - Índice Real (base 1994 = 100)': 'Real Minimum Wage (Index)',
    'Cotizaciones de productos (promedio del periodo) - Trigo - EEUU (US$ por toneladas)': 'Wheat (US$ per ton)',
    'Cotizaciones de productos (promedio del periodo) - Maíz - EEUU (US$ por toneladas)': 'Corn  (US$ per ton)',
    'Cotizaciones de productos (promedio del periodo) - Aceite Soya - EEUU (US$ por toneladas)': 'Soybean oil (US$ per ton)',
    'Cotizaciones de productos (promedio del periodo) - Petróleo - WTI (US$ por barriles)': 'Crude oil (US$ per barrel)'  
}

# We rename the columns so they are easier to analyse
df.rename(columns=columns, inplace=True)

In [15]:
df_lags = df.copy()

for variable in df_lags.columns[1:]:
    df_lags[f'{variable}_lag_1'] = df_lags[variable].shift()
    df_lags[f'{variable}_lag_2'] = df_lags[variable].shift(2)

In [16]:
# We delete contemporary variables
df_lags.drop(columns = ['CPI Tradable', 'CPI Non-Tradable', 'CPI', 'CPI Non-Core', 'CPI Food and Energy', 'CPI Excluding Food and Energy',
       'CPI Food and Beverages', 'CPI Excluding Food and Beverages','CPI Core Excluding Food and Beverages', 'CPI Imported',
       'Wholesale Price Index', 'Reserve Requirement Rate','Monetary Policy Rate','Circulating Currency Seasonally Adjusted (mill S/)',
       'Net International Reserves (mill $)', 'Real Minimum Wage (Index)', 'Wheat (US$ per ton)', 'Corn  (US$ per ton)', 
       'Soybean oil (US$ per ton)', 'Crude oil (US$ per barrel)'], inplace = True)

df_lags = df_lags.dropna()

## 1.5 Save Results
We save it to the ```input``` folder, where we can use it to do the forecasting in the next notebook.

In [18]:
df.to_csv('../../../input/df_raw_c08.csv')

In [19]:
df_lags.to_csv('../../../input/df_lags_c08.csv')

In [20]:
df_lags.tail()

,CPI Core,CPI Tradable_lag_1,CPI Tradable_lag_2,CPI Non-Tradable_lag_1,CPI Non-Tradable_lag_2,CPI_lag_1,CPI_lag_2,CPI Non-Core_lag_1,CPI Non-Core_lag_2,CPI Food and Energy_lag_1,...,Real Minimum Wage (Index)_lag_1,Real Minimum Wage (Index)_lag_2,Wheat (US$ per ton)_lag_1,Wheat (US$ per ton)_lag_2,Corn (US$ per ton)_lag_1,Corn (US$ per ton)_lag_2,Soybean oil (US$ per ton)_lag_1,Soybean oil (US$ per ton)_lag_2,Crude oil (US$ per barrel)_lag_1,Crude oil (US$ per barrel)_lag_2
Fecha,,,,,,,,,,,,,,,,,,,,,
2008-08-01,0.469680,0.375112,0.393584,0.658619,0.626801,0.542800,0.531498,0.656495,0.646992,0.683126,...,-0.005524,-0.007637,-0.059006,0.021662,-0.085210,0.169325,-0.030379,0.055625,0.000075,0.065257
2008-09-01,0.481624,0.356039,0.375112,0.690723,0.658619,0.554025,0.542800,0.666096,0.656495,0.681738,...,-0.005870,-0.005524,-0.012197,-0.059006,-0.160701,-0.085210,-0.156075,-0.030379,-0.128561,0.000075
2008-10-01,0.493295,0.336413,0.356039,0.723117,0.690723,0.565199,0.554025,0.675909,0.666096,0.680067,...,-0.005640,-0.005870,-0.125494,-0.012197,-0.006954,-0.160701,-0.078231,-0.156075,-0.103871,-0.128561
2008-11-01,0.504673,0.316247,0.336413,0.755818,0.723117,0.576337,0.565199,0.686014,0.675909,0.678143,...,-0.006094,-0.005640,-0.206207,-0.125494,-0.260785,-0.006954,-0.244475,-0.078231,-0.267189,-0.103871
2008-12-01,0.515746,0.295514,0.316247,0.788926,0.755818,0.587490,0.576337,0.696555,0.686014,0.676114,...,-0.003081,-0.006094,0.021940,-0.206207,-0.090908,-0.260785,-0.102564,-0.244475,-0.253950,-0.267189


In [21]:
df.tail()

,CPI Core,CPI Tradable,CPI Non-Tradable,CPI,CPI Non-Core,CPI Food and Energy,CPI Excluding Food and Energy,CPI Food and Beverages,CPI Excluding Food and Beverages,CPI Core Excluding Food and Beverages,...,Wholesale Price Index,Reserve Requirement Rate,Monetary Policy Rate,Circulating Currency Seasonally Adjusted (mill S/),Net International Reserves (mill $),Real Minimum Wage (Index),Wheat (US$ per ton),Corn (US$ per ton),Soybean oil (US$ per ton),Crude oil (US$ per barrel)
Fecha,,,,,,,,,,,,,,,,,,,,,
2008-08-01,0.469680,0.356039,0.690723,0.554025,0.666096,0.681738,0.394014,0.755351,0.365874,0.317970,...,0.702300,11.472876,6.25,0.005009,0.002124,-0.005870,-0.012197,-0.160701,-0.156075,-0.128561
2008-09-01,0.481624,0.336413,0.723117,0.565199,0.675909,0.680067,0.422250,0.754096,0.388973,0.335349,...,0.693072,10.787601,6.50,0.017596,-0.006157,-0.005640,-0.125494,-0.006954,-0.078231,-0.103871
2008-10-01,0.493295,0.316247,0.755818,0.576337,0.686014,0.678143,0.450790,0.752209,0.412633,0.352862,...,0.682759,11.862506,6.50,0.003966,-0.079794,-0.006094,-0.206207,-0.260785,-0.244475,-0.267189
2008-11-01,0.504673,0.295514,0.788926,0.587490,0.696555,0.676114,0.479560,0.749865,0.436787,0.370484,...,0.671225,10.084114,6.50,0.006653,-0.030157,-0.003081,0.021940,-0.090908,-0.102564,-0.253950
2008-12-01,0.515746,0.274156,0.822524,0.598682,0.707608,0.674121,0.508447,0.747301,0.461270,0.388144,...,0.658290,12.516334,6.50,-0.005561,0.007297,-0.003589,-0.012756,-0.010237,-0.077524,-0.287431
